In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Diseño del Score Global de riesgo VereData V1.0

En este notebook se encuentra el procedimiento para el diseño de la primera versión del score global de riesgo de VereData. Este indicador esta compuesto por 40 variles dividida en cinco compoentes: contexto socioeconómico, infraestructura, criminalidad, conflcito armado y ambiental. (Ver ilustración 1)

Las variables tienen periodicidad anual, y se usa su último año disponible (En algunos casos es necesario actualizarlas).

Nota: al importar la base se elimina la primera columna que no tiene información, y los ROS de Bancolombia, porque es información confidencial.

In [2]:
df = pd.read_csv("variables.csv", converters={'coddane':str})

In [3]:
df = df.drop(['Unnamed: 0', 'num_ros_perc'], axis=1)

## Transformación de variables:

Las variables pasan por un proceso de transformación para reducir el efecto de factores externos sobre ellas que pueden dificultar el análisis del indicador. 

Por ejemplo, algunos municipios tienen una población muy grande en comparación a los demás, por lo que es normal que sucedan una mayor cantidad de delitos, por lo tanto, es necesario transformarlos a tasas por cierta cantidad de habitantes, para poder compararlo con los demás municipios.

Más adelante se pondrá una tabla señalando que transformaciones se hicieron a qué variables.

 - Transformación a tasas: algunas variables relacionadas con la población se transformaron a tasas para mejorar la capacidad de análisis del score y comparación entre los municipios, las de delitos y otras variables relacionadas con la población se tranformaron en tasas de cada 100.000 habitantes, mientras las que se relacionaban mejor con la extensión geográfica se tranformo en tasas por hectareas.
 
 - Transformación de dummies a continuas: cuandos se normaliza una variable dummy no cambia su valor, dandole la calificación máxima a los municipios que tienen 1, y la mínima a los que tienen 0. Estas    calificaciones tienen mucho peso sobre el indicador hasta el punto de sesgar sus resultados.
 
 - Transformación de dummies: las variables dummies tienen valores fijos de 1 y 0 que al normalizar la seríe le da la máxima puntuación a los municipios con valores 1 y la mínima a los de 0. Esto causaba que estas variables tuvieron un peso mayor en el score, comparación con las variables continuas, por tal motivo, se hicieron aproximaciones con valor agregado.


In [4]:
#transformación de dummies y discretas
df['fronterizo_x_va'] = df['mpio_fronterizo']*df['va_ppa_2019']
df['pasofr_x_va'] =  df['paso_fr']*df['va_ppa_2019']
df['puertos_x_va'] =  df['puerto_libre']*df['va_ppa_2019']
df['zf_x_va'] = df['suma_zf']*df['va_ppa_2019']
df['rios_x_ha'] = df['presencia_rios']/df['ha_municipal']

In [5]:
#creación de proxys
df['desigualdad'] = df['ipm_2018']*df['va_ppa_2019']

In [6]:
df.columns

Index(['coddane', 'MPIO_y', 'DPNOM_x', 'gobierno_abierto_2016',
       'nbi_total_2018', 'mpio_fronterizo', 'riesgo_victimizacion_2020',
       'coca19_HA_perc_area', 'produccion_oro_2020_perc',
       'numero_grupos_armados', 'acto_terrorista_2019_perc',
       'desaparicion_2019_perc', 'desplazamiento_2019_perc',
       'tasa_homicidios_2017', 'minas_antipersonales_2020_perc',
       'tasa_extorsion_2020_perc', 'eova_ilicita_perc_area', 'ipm_2018',
       'aerodromos_perc', 'mdm_2019', 'va_ppa_2019',
       'maquinaria_incautada_2020_perc', 'tasa_desempleo_2019',
       'ingresos_juegos_2019_perc', 'armamento_incautado_2020_perc',
       'cocaina_incautada_2020_perc', 'marihuana_incautada_2020_perc',
       'capturados_contrabando_perc', 'cigarrillos_incautados_2017_perc',
       'capturas_lavado_perc', 'presencia_rios', 'dumm_coca19_HA',
       'dumm_produccion_oro_2020', 'dumm_acto_terrorista_2019',
       'dumm_desaparicion_2019', 'dumm_desplazamiento_2019',
       'dumm_minas_ant

In [7]:
#poner negativas las variables que tienen una relación inversa de riesgo, para que la normalización sea inversa
negativas = ['mdm_2019','va_ppa_2019','gobierno_abierto_2016']

df[negativas] *= -1

In [8]:
contexto = ['coddane','gobierno_abierto_2016', 'produccion_oro_2020_perc',
            'fronterizo_x_va', 'mdm_2019','ingresos_juegos_2019_perc','tasa_desempleo_2019','desigualdad']

In [9]:
df1 = df[contexto]

In [10]:
df1.head()

,coddane,gobierno_abierto_2016,produccion_oro_2020_perc,fronterizo_x_va,mdm_2019,ingresos_juegos_2019_perc,tasa_desempleo_2019,desigualdad
0,05001,-85.594530,0.0,0.000000,-83.595858,0.000000e+00,0.113831,309.105540
1,11001,-85.008696,0.0,0.000000,-79.632308,7.221464e+08,0.109023,289.484364
2,54001,-76.967960,0.0,12.032173,-67.903241,0.000000e+00,0.141324,309.226857
3,08001,-79.552880,0.0,0.000000,-72.386381,1.098686e+07,0.070208,388.748826
4,76001,-74.387320,0.0,0.000000,-83.744096,4.439283e+00,0.119149,232.685484


In [11]:
#normalizar de 0 a 1 con método de mínimos y máximos
scaler = preprocessing.MinMaxScaler()
names = df1.columns
cols = df1.columns.difference(['coddane'])
d = scaler.fit_transform(df1[cols])
sc_df1 = pd.DataFrame(d, columns=cols)
sc_df1.head()

,desigualdad,fronterizo_x_va,gobierno_abierto_2016,ingresos_juegos_2019_perc,mdm_2019,produccion_oro_2020_perc,tasa_desempleo_2019
0,0.017711,0.000000,0.058905,0.000000e+00,0.037102,0.0,0.737245
1,0.016122,0.000000,0.065346,3.098261e-01,0.082756,0.0,0.706111
2,0.017721,0.115617,0.153752,0.000000e+00,0.217857,0.0,0.915311
3,0.024160,0.000000,0.125331,4.713746e-03,0.166218,0.0,0.454714
4,0.011522,0.000000,0.182126,1.904608e-09,0.035394,0.0,0.771691


In [12]:
#calcular la media de las filas
sc_df1['score_cs'] = sc_df1.mean(axis=1)

In [13]:
#unir municipios con sus valores
sc_df1 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc_df1], axis = 1)
sc_df1['score_cs']=sc_df1['score_cs'].round(decimals = 3)
sc_df1.head()

,coddane,MPIO_y,DPNOM_x,desigualdad,fronterizo_x_va,gobierno_abierto_2016,ingresos_juegos_2019_perc,mdm_2019,produccion_oro_2020_perc,tasa_desempleo_2019,score_cs
0,05001,Medellín,Antioquia,0.017711,0.000000,0.058905,0.000000e+00,0.037102,0.0,0.737245,0.122
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.016122,0.000000,0.065346,3.098261e-01,0.082756,0.0,0.706111,0.169
2,54001,San José de Cúcuta,Norte de Santander,0.017721,0.115617,0.153752,0.000000e+00,0.217857,0.0,0.915311,0.203
3,08001,Barranquilla,Atlántico,0.024160,0.000000,0.125331,4.713746e-03,0.166218,0.0,0.454714,0.111
4,76001,Cali,Valle del Cauca,0.011522,0.000000,0.182126,1.904608e-09,0.035394,0.0,0.771691,0.143


In [14]:
#dataframe con el score
cs = sc_df1[['coddane','score_cs']]

#volver a normalizar
scaler = preprocessing.MinMaxScaler()
names = cs.columns
cols = cs.columns.difference(['coddane'])
d = scaler.fit_transform(cs[cols])
sc2_df1 = pd.DataFrame(d, columns=cols)
sc2_df1.head()

,score_cs
0,0.191950
1,0.337461
2,0.442724
3,0.157895
4,0.256966


## Unir variables

In [15]:
#unir nuevamente para construir el df
sc2_df1 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc2_df1], axis = 1)
cs = sc2_df1[['coddane', 'MPIO_y','DPNOM_x','score_cs']]

In [16]:
#ranking componente de contexto socioeconómico
cs.sort_values(by = 'score_cs', ascending=False)

,coddane,MPIO_y,DPNOM_x,score_cs
312,20045,Becerril,Cesar,1.000000
527,05113,Buriticá,Antioquia,0.978328
995,27810,Unión Panamericana,Chocó,0.869969
166,44078,Barrancas,La Guajira,0.863777
238,05604,Remedios,Antioquia,0.860681
...,...,...,...,...
99,95001,San José del Guaviare,Guaviare,0.049536
161,85139,Maní,Casanare,0.027864
20,85001,Yopal,Casanare,0.009288
229,85162,Monterrey,Casanare,0.006192


Infraestructura

In [17]:
infraestructura =['coddane', 'rios_x_ha', 'maq_ilegalmin_2021','aerodromos_perc','vol_oleductos_2021','vol_torres_2018','zf_x_va','puertos_x_va','pasofr_x_va']

In [18]:
df2 = df[infraestructura]

In [19]:
scaler = preprocessing.MinMaxScaler()
names = df2.columns
cols = df2.columns.difference(['coddane'])
d = scaler.fit_transform(df2[cols])
sc_df2 = pd.DataFrame(d, columns=cols)
sc_df2.head()

,aerodromos_perc,maq_ilegalmin_2021,pasofr_x_va,puertos_x_va,rios_x_ha,vol_oleductos_2021,vol_torres_2018,zf_x_va
0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.116861
1,0.001605,0.108844,0.0,0.0,0.000000,0.0,0.0,0.311303
2,0.010663,0.000000,1.0,0.0,0.000000,0.0,0.0,0.116452
3,0.000000,0.006803,0.0,0.0,0.272686,0.0,0.0,0.324349
4,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.094622


In [20]:
#calcular la media de las filas
sc_df2['score_inf'] = sc_df2.mean(axis=1)

In [21]:
#unir municipios con sus valores
sc_df2 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc_df2], axis = 1)
sc_df2['score_inf']=sc_df2['score_inf'].round(decimals = 3)
sc_df2.head()

,coddane,MPIO_y,DPNOM_x,aerodromos_perc,maq_ilegalmin_2021,pasofr_x_va,puertos_x_va,rios_x_ha,vol_oleductos_2021,vol_torres_2018,zf_x_va,score_inf
0,05001,Medellín,Antioquia,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.116861,0.015
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.001605,0.108844,0.0,0.0,0.000000,0.0,0.0,0.311303,0.053
2,54001,San José de Cúcuta,Norte de Santander,0.010663,0.000000,1.0,0.0,0.000000,0.0,0.0,0.116452,0.141
3,08001,Barranquilla,Atlántico,0.000000,0.006803,0.0,0.0,0.272686,0.0,0.0,0.324349,0.075
4,76001,Cali,Valle del Cauca,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.094622,0.012


In [22]:
#dataframe con el score
inf = sc_df2[['coddane','score_inf']]

#volver a normalizar
scaler = preprocessing.MinMaxScaler()
names = inf.columns
cols = inf.columns.difference(['coddane'])
d = scaler.fit_transform(inf[cols])
sc2_df2 = pd.DataFrame(d, columns=cols)
sc2_df2.head()

,score_inf
0,0.072115
1,0.254808
2,0.677885
3,0.360577
4,0.057692


In [23]:
#unir nuevamente para construir el df
sc2_df2 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc2_df2], axis = 1)
inf = sc2_df2[['coddane', 'MPIO_y','DPNOM_x','score_inf']]

In [24]:
#ranking componente de infraestructura
inf.sort_values(by = 'score_inf', ascending=False)

,coddane,MPIO_y,DPNOM_x,score_inf
973,88564,Providencia,Archipiélago de San Andrés,1.000000
2,54001,San José de Cúcuta,Norte de Santander,0.677885
363,81736,Saravena,Arauca,0.668269
47,52835,San Andrés de Tumaco,Nariño,0.634615
311,05495,Nechí,Antioquia,0.629808
...,...,...,...,...
602,15542,Pesca,Boyacá,0.000000
603,47545,Pijiño del Carmen,Magdalena,0.000000
604,81300,Fortul,Arauca,0.000000
220,17873,Villamaría,Caldas,0.000000


Criminalidad

In [25]:
criminalidad = ['coddane','coca19_HA_perc_area','eova_ilicita_perc_area','cigarrillos_incautados_2017_perc',
               'capturados_contrabando_perc','capturas_lavado_perc','tasa_homicidios_2017',
                'marihuana_incautada_2020_perc','cocaina_incautada_2020_perc','hurtosc_perc_2021',
                'hurtosp_perc_2021','sae_2021','riesgo_electoral_2018']

df3 = df[criminalidad]

In [26]:
scaler = preprocessing.MinMaxScaler()
names = df3.columns
cols = df3.columns.difference(['coddane'])
d = scaler.fit_transform(df3[cols])
sc_df3 = pd.DataFrame(d, columns=cols)
sc_df3.head()

,capturados_contrabando_perc,capturas_lavado_perc,cigarrillos_incautados_2017_perc,coca19_HA_perc_area,cocaina_incautada_2020_perc,eova_ilicita_perc_area,hurtosc_perc_2021,hurtosp_perc_2021,marihuana_incautada_2020_perc,riesgo_electoral_2018,sae_2021,tasa_homicidios_2017
0,0.000000,0.0,0.000445,0.000000,0.000295,0.0,0.387831,0.654431,0.000972,0.0,0.033333,0.084944
1,0.026177,0.0,0.000053,0.000000,0.000332,0.0,0.413356,1.000000,0.002451,1.0,0.000000,0.052338
2,0.298121,0.0,0.001705,0.028628,0.000088,0.0,0.198538,0.269389,0.000962,1.0,0.000000,0.140583
3,0.056816,0.0,0.001679,0.000000,0.004783,0.0,0.320130,0.585157,0.000515,0.0,0.000000,0.107207
4,0.025711,0.0,0.000432,0.000000,0.000434,0.0,0.493570,0.596615,0.020747,1.0,0.096296,0.188080


In [27]:
#calcular la media de las filas
sc_df3['score_cri'] = sc_df3.mean(axis=1)

In [28]:
#unir municipios con sus valores
sc_df3 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc_df3], axis = 1)
sc_df3['score_cri']=sc_df3['score_cri'].round(decimals = 3)
sc_df3.head()

,coddane,MPIO_y,DPNOM_x,capturados_contrabando_perc,capturas_lavado_perc,cigarrillos_incautados_2017_perc,coca19_HA_perc_area,cocaina_incautada_2020_perc,eova_ilicita_perc_area,hurtosc_perc_2021,hurtosp_perc_2021,marihuana_incautada_2020_perc,riesgo_electoral_2018,sae_2021,tasa_homicidios_2017,score_cri
0,05001,Medellín,Antioquia,0.000000,0.0,0.000445,0.000000,0.000295,0.0,0.387831,0.654431,0.000972,0.0,0.033333,0.084944,0.097
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.026177,0.0,0.000053,0.000000,0.000332,0.0,0.413356,1.000000,0.002451,1.0,0.000000,0.052338,0.208
2,54001,San José de Cúcuta,Norte de Santander,0.298121,0.0,0.001705,0.028628,0.000088,0.0,0.198538,0.269389,0.000962,1.0,0.000000,0.140583,0.162
3,08001,Barranquilla,Atlántico,0.056816,0.0,0.001679,0.000000,0.004783,0.0,0.320130,0.585157,0.000515,0.0,0.000000,0.107207,0.090
4,76001,Cali,Valle del Cauca,0.025711,0.0,0.000432,0.000000,0.000434,0.0,0.493570,0.596615,0.020747,1.0,0.096296,0.188080,0.202


In [29]:
#dataframe con el score
cri = sc_df3[['coddane','score_cri']]

#volver a normalizar
scaler = preprocessing.MinMaxScaler()
names = cri.columns
cols = cri.columns.difference(['coddane'])
d = scaler.fit_transform(cri[cols])
sc2_df3 = pd.DataFrame(d, columns=cols)
sc2_df3.head()

,score_cri
0,0.411017
1,0.881356
2,0.686441
3,0.381356
4,0.855932


In [30]:
#unir nuevamente para construir el df
sc2_df3 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc2_df3], axis = 1)
cri = sc2_df3[['coddane', 'MPIO_y','DPNOM_x','score_cri']]

In [31]:
#ranking componente de criminalidad
cri.sort_values(by = 'score_cri', ascending=False)

,coddane,MPIO_y,DPNOM_x,score_cri
210,54810,Tibú,Norte de Santander,1.000000
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.881356
769,54250,El Tarra,Norte de Santander,0.881356
4,76001,Cali,Valle del Cauca,0.855932
26,19001,Popayán,Cauca,0.851695
...,...,...,...,...
830,15114,Busbanzá,Boyacá,0.000000
820,15832,Tununguá,Boyacá,0.000000
1034,15276,Floresta,Boyacá,0.000000
786,68673,San Benito,Santander,0.000000


Conflicto armado

In [32]:
df.columns

Index(['coddane', 'MPIO_y', 'DPNOM_x', 'gobierno_abierto_2016',
       'nbi_total_2018', 'mpio_fronterizo', 'riesgo_victimizacion_2020',
       'coca19_HA_perc_area', 'produccion_oro_2020_perc',
       'numero_grupos_armados', 'acto_terrorista_2019_perc',
       'desaparicion_2019_perc', 'desplazamiento_2019_perc',
       'tasa_homicidios_2017', 'minas_antipersonales_2020_perc',
       'tasa_extorsion_2020_perc', 'eova_ilicita_perc_area', 'ipm_2018',
       'aerodromos_perc', 'mdm_2019', 'va_ppa_2019',
       'maquinaria_incautada_2020_perc', 'tasa_desempleo_2019',
       'ingresos_juegos_2019_perc', 'armamento_incautado_2020_perc',
       'cocaina_incautada_2020_perc', 'marihuana_incautada_2020_perc',
       'capturados_contrabando_perc', 'cigarrillos_incautados_2017_perc',
       'capturas_lavado_perc', 'presencia_rios', 'dumm_coca19_HA',
       'dumm_produccion_oro_2020', 'dumm_acto_terrorista_2019',
       'dumm_desaparicion_2019', 'dumm_desplazamiento_2019',
       'dumm_minas_ant

In [34]:
conflicto = ['coddane', 'numero_grupos_armados','acto_terrorista_2019_perc','tasa_extorsion_2020_perc',
            'riesgo_victimizacion_2020','desaparicion_2019_perc','desplazamiento_2019_perc',
             'minas_antipersonales_2020_perc','armamento_incautado_2020_perc','solicitudes_tierras_2022','gpa_r_y']

df4 = df[conflicto]

In [35]:
scaler = preprocessing.MinMaxScaler()
names = df4.columns
cols = df4.columns.difference(['coddane'])
d = scaler.fit_transform(df4[cols])
sc_df4 = pd.DataFrame(d, columns=cols)
sc_df4.head()

,acto_terrorista_2019_perc,armamento_incautado_2020_perc,desaparicion_2019_perc,desplazamiento_2019_perc,gpa_r_y,minas_antipersonales_2020_perc,numero_grupos_armados,riesgo_victimizacion_2020,solicitudes_tierras_2022,tasa_extorsion_2020_perc
0,0.000000,0.002521,0.000000,0.002192,0.0,0.000000,0.444444,0.191146,0.000000,0.000096
1,0.001578,0.000433,0.000000,0.000084,0.0,0.000000,0.000000,0.131234,0.147929,0.000021
2,0.000000,0.005548,0.007501,0.002793,0.0,0.003368,0.111111,0.193334,0.274951,0.000269
3,0.000000,0.000000,0.000000,0.000270,0.0,0.000000,0.000000,0.110527,0.000000,0.000091
4,0.000000,0.001205,0.000000,0.000762,0.0,0.000000,0.055556,0.188152,0.121893,0.000128


In [36]:
#calcular la media de las filas
sc_df4['score_con'] = sc_df4.mean(axis=1)

In [37]:
#unir municipios con sus valores
sc_df4 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc_df4], axis = 1)
sc_df4['score_con']=sc_df4['score_con'].round(decimals = 3)
sc_df4.head()

,coddane,MPIO_y,DPNOM_x,acto_terrorista_2019_perc,armamento_incautado_2020_perc,desaparicion_2019_perc,desplazamiento_2019_perc,gpa_r_y,minas_antipersonales_2020_perc,numero_grupos_armados,riesgo_victimizacion_2020,solicitudes_tierras_2022,tasa_extorsion_2020_perc,score_con
0,05001,Medellín,Antioquia,0.000000,0.002521,0.000000,0.002192,0.0,0.000000,0.444444,0.191146,0.000000,0.000096,0.064
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.001578,0.000433,0.000000,0.000084,0.0,0.000000,0.000000,0.131234,0.147929,0.000021,0.028
2,54001,San José de Cúcuta,Norte de Santander,0.000000,0.005548,0.007501,0.002793,0.0,0.003368,0.111111,0.193334,0.274951,0.000269,0.060
3,08001,Barranquilla,Atlántico,0.000000,0.000000,0.000000,0.000270,0.0,0.000000,0.000000,0.110527,0.000000,0.000091,0.011
4,76001,Cali,Valle del Cauca,0.000000,0.001205,0.000000,0.000762,0.0,0.000000,0.055556,0.188152,0.121893,0.000128,0.037


In [38]:
sc_df4.shape

(1122, 14)

In [39]:
sc_df4['coddane'].nunique()

1121

In [40]:
sc_df4[sc_df4['coddane']=='05266'].transpose()

,34
coddane,05266
MPIO_y,Envigado
DPNOM_x,Antioquia
acto_terrorista_2019_perc,0.0
armamento_incautado_2020_perc,0.0
desaparicion_2019_perc,0.0
desplazamiento_2019_perc,0.0
gpa_r_y,0.0
minas_antipersonales_2020_perc,0.0
numero_grupos_armados,0.055556


In [41]:
#dataframe con el score
con = sc_df4[['coddane','score_con']]

#volver a normalizar
scaler = preprocessing.MinMaxScaler()
names = con.columns
cols = con.columns.difference(['coddane'])
d = scaler.fit_transform(con[cols])
sc2_df4 = pd.DataFrame(d, columns=cols)
sc2_df4.head()

,score_con
0,0.166234
1,0.072727
2,0.155844
3,0.028571
4,0.096104


In [42]:
#unir nuevamente para construir el df
sc2_df4 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc2_df4], axis = 1)
con = sc2_df4[['coddane', 'MPIO_y','DPNOM_x','score_con']]

In [43]:
#ranking componente de conflicto armado
con.sort_values(by = 'score_con', ascending=False)

,coddane,MPIO_y,DPNOM_x,score_con
278,05790,Tarazá,Antioquia,1.000000
965,54344,Hacarí,Norte de Santander,0.880519
1004,54800,Teorama,Norte de Santander,0.859740
47,52835,San Andrés de Tumaco,Nariño,0.818182
210,54810,Tibú,Norte de Santander,0.800000
...,...,...,...,...
983,15522,Panqueba,Boyacá,0.002597
935,25299,Gama,Cundinamarca,0.002597
905,25372,Junín,Cundinamarca,0.002597
1059,15317,Guacamayas,Boyacá,0.000000


Ambiental

In [44]:
ambiental = ['irac_2018','ha_deforestadas_2016'] 
df5 = df[ambiental]

In [45]:
#scaler = preprocessing.MinMaxScaler()
names = df5.columns
cols = df5.columns.difference(['coddane'])
d = scaler.fit_transform(df5[cols])
sc_df5 = pd.DataFrame(d, columns=cols)
sc_df5.head()

,ha_deforestadas_2016,irac_2018
0,0.002924,0.187741
1,0.000092,0.102808
2,0.017244,0.378487
3,0.000202,0.181618
4,0.000215,0.295097


In [46]:
#calcular la media de las filas
sc_df5['score_amb'] = sc_df5.mean(axis=1)

In [47]:
#unir municipios con sus valores
sc_df5 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc_df5], axis = 1)
sc_df5['score_amb']=sc_df5['score_amb'].round(decimals = 3)
sc_df5.head()

,coddane,MPIO_y,DPNOM_x,ha_deforestadas_2016,irac_2018,score_amb
0,05001,Medellín,Antioquia,0.002924,0.187741,0.095
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.000092,0.102808,0.051
2,54001,San José de Cúcuta,Norte de Santander,0.017244,0.378487,0.198
3,08001,Barranquilla,Atlántico,0.000202,0.181618,0.091
4,76001,Cali,Valle del Cauca,0.000215,0.295097,0.148


In [48]:
#dataframe con el score
amb = sc_df5[['coddane','score_amb']]

#volver a normalizar
scaler = preprocessing.MinMaxScaler()
names = amb.columns
cols = amb.columns.difference(['coddane'])
d = scaler.fit_transform(amb[cols])
sc2_df5 = pd.DataFrame(d, columns=cols)
sc2_df5.head()

,score_amb
0,0.110981
1,0.059579
2,0.231308
3,0.106308
4,0.172897


In [49]:
#unir nuevamente para construir el df
sc2_df5 = pd.concat([df[['coddane', 'MPIO_y', 'DPNOM_x']],sc2_df5], axis = 1)
amb = sc2_df5[['coddane', 'MPIO_y','DPNOM_x','score_amb']]

In [50]:
#ranking componente de ambiental
amb.sort_values(by = 'score_amb', ascending=False)

,coddane,MPIO_y,DPNOM_x,score_amb
375,27615,Riosucio,Chocó,1.000000
737,27800,Unguía,Chocó,0.853972
41,18753,San Vicente del Caguán,Caquetá,0.831776
685,50370,Uribe,Meta,0.761682
319,18150,Cartagena del Chairá,Caquetá,0.733645
...,...,...,...,...
1,11001,"Bogotá, D.C.","Bogotá, D.C.",0.059579
157,08573,Puerto Colombia,Atlántico,0.058411
229,85162,Monterrey,Casanare,0.054907
228,50150,Castilla la Nueva,Meta,0.031542


Score global VereData

In [51]:
#se unen los componentes
score_global = cs.merge(inf, on=['coddane','MPIO_y','DPNOM_x'], how='left')
score_global = score_global.merge(cri, on=['coddane','MPIO_y','DPNOM_x'], how='left')
score_global = score_global.merge(con, on=['coddane','MPIO_y','DPNOM_x'], how='left')
score_global = score_global.merge(amb, on=['coddane','MPIO_y','DPNOM_x'], how='left')

In [52]:
score_global['score_global']=score_global[['score_cs','score_inf','score_cri','score_con','score_amb']].mean(axis=1)

In [53]:
sg_ranking = score_global.sort_values(by='score_global', ascending =False)
sg_ranking.head(20)

,coddane,MPIO_y,DPNOM_x,score_cs,score_inf,score_cri,score_con,score_amb,score_global
210,54810,Tibú,Norte de Santander,0.625387,0.245192,1.000000,0.800000,0.556075,0.645331
47,52835,San Andrés de Tumaco,Nariño,0.433437,0.634615,0.686441,0.818182,0.337617,0.582058
769,54250,El Tarra,Norte de Santander,0.557276,0.000000,0.881356,0.657143,0.664720,0.552099
278,05790,Tarazá,Antioquia,0.619195,0.413462,0.237288,1.000000,0.375000,0.528989
1034,54800,Teorama,Norte de Santander,0.541796,0.000000,0.648305,0.859740,0.417056,0.493379
249,05854,Valdivia,Antioquia,0.464396,0.447115,0.466102,0.566234,0.431075,0.474984
995,54344,Hacarí,Norte de Santander,0.501548,0.000000,0.504237,0.880519,0.413551,0.459971
851,54670,San Calixto,Norte de Santander,0.541796,0.000000,0.402542,0.766234,0.511682,0.444451
2,54001,San José de Cúcuta,Norte de Santander,0.442724,0.677885,0.686441,0.155844,0.231308,0.438840
1112,27660,San José del Palmar,Chocó,0.455108,0.000000,0.716102,0.464935,0.542056,0.435640


In [54]:
sg_ranking.tail(20)

,coddane,MPIO_y,DPNOM_x,score_cs,score_inf,score_cri,score_con,score_amb,score_global
283,15491,Nobsa,Boyacá,0.340557,0.000000,0.097458,0.023377,0.084112,0.109101
366,85263,Pore,Casanare,0.086687,0.000000,0.148305,0.075325,0.227804,0.107624
116,85410,Tauramena,Casanare,0.102167,0.009615,0.135593,0.090909,0.195093,0.106676
629,13222,Clemencia,Bolívar,0.275542,0.000000,0.050847,0.033766,0.172897,0.106611
114,85440,Villanueva,Casanare,0.102167,0.028846,0.148305,0.049351,0.195093,0.104753
1044,97161,Carurú,Vaupés,0.126935,0.004808,0.016949,0.174026,0.197430,0.104030
726,52210,Contadero,Nariño,0.213622,0.000000,0.021186,0.015584,0.267523,0.103583
495,15367,Jenesano,Boyacá,0.229102,0.000000,0.021186,0.012987,0.242991,0.101253
938,68121,Cabrera,Santander,0.284830,0.000000,0.000000,0.010390,0.207944,0.100633
1085,15187,Chivatá,Boyacá,0.244582,0.000000,0.042373,0.018182,0.197430,0.100513


# Reportes

In [55]:
def reporte_componente (coddane, sc_dfx):
    """Retorna reporte por componente del municipio
        Parámetros: 
            coddane (str): Código Dane del municipio de 5 dígitos
            sc_dfx : dataframes por componentes, donde 1 es contexto, 2 es infraestructura, 3 criminalidad, 4 conflicto y 5 ambiental
            
            """
    #sg_base=sg_base[componente]
    
    reporte_componente = pd.concat([sc_dfx[sc_dfx['coddane']==coddane].transpose(), sc_dfx.mean()], axis=1)
        
    reporte_componente.columns = ['municipio', 'media_nacional']
    
    reporte_componente.drop(['coddane'], inplace=True)

    reporte_componente.reset_index(inplace=True)
    
    return reporte_componente

In [56]:
reporte_componente('05266',sc_df3)

,index,municipio,media_nacional
0,MPIO_y,Envigado,NaN
1,DPNOM_x,Antioquia,NaN
2,capturados_contrabando_perc,0.0,0.002823
3,capturas_lavado_perc,0.0,0.001394
4,cigarrillos_incautados_2017_perc,0.012394,0.004197
5,coca19_HA_perc_area,0.0,0.011152
6,cocaina_incautada_2020_perc,0.000004,0.009003
7,eova_ilicita_perc_area,0.0,0.005225
8,hurtosc_perc_2021,0.423005,0.109558
9,hurtosp_perc_2021,0.451059,0.081779


In [57]:
reporte_componente('50150',sc_df2)

,index,municipio,media_nacional
0,MPIO_y,Castilla la Nueva,NaN
1,DPNOM_x,Meta,NaN
2,aerodromos_perc,0.0,0.009641
3,maq_ilegalmin_2021,0.0,0.006299
4,pasofr_x_va,0.0,0.003062
5,puertos_x_va,0.0,0.002038
6,rios_x_ha,0.0,0.017228
7,vol_oleductos_2021,0.0,0.002350
8,vol_torres_2018,0.0,0.003565
9,zf_x_va,0.0,0.008578


In [ ]:
ranking_crim = sc_df3.sort_values(by='score_cri',ascending=False)
ranking_crim.head(20)


In [ ]:
ranking_crim.tail(20)